# Presentation Notebook

This notebook is meant to be used as the main "put it all together" notebook which we can use to present the models we have worked on.

In [1]:
SHOW_INTERMEDIATE_DATA = True # set this option if you want to display the data at each step, not just the final result.

### Setup: import libraries and files

In [2]:
import pandas as pd
import numpy as np
from MonteCarlo.monte_carlo import montecarlo as mc
from HybridModel.hybrid_model import hybrid_model as hm
from BlackScholes.black_scholes import optionPrice as bs

In [3]:
AMD_2019 = pd.read_csv("Data/AMD/AMD_2019.csv")
AMD_2020 = pd.read_csv("Data/AMD/AMD_2020.csv")
AMD_2021 = pd.read_csv("Data/AMD/AMD_2021.csv")
AMD_2022 = pd.read_csv("Data/AMD/AMD_2022.csv")
AMD_2023 = pd.read_csv("Data/AMD/AMD_2023.csv")
AMD_5_yrs = pd.concat([AMD_2019, AMD_2020, AMD_2021, AMD_2022, AMD_2023]).reset_index()
AMD_3_yrs = pd.concat([AMD_2021, AMD_2022, AMD_2023]).reset_index() 

df = AMD_3_yrs # replace this with any of the defined dataframes above to use their data. (i.e. you could replace this with other stock data) We are just using AMD.
# df = df[:209]

### Calculate Volatility Off Historic Stock Prices

In [4]:
if SHOW_INTERMEDIATE_DATA:
    display(df[["date", "open", "close", "high", "low"]])

,date,open,close,high,low
0,2019-01-02,18.05,18.82,19.00,17.98
1,2019-01-03,18.39,17.05,18.68,16.94
2,2019-01-04,17.57,19.00,19.07,17.43
3,2019-01-07,19.43,20.57,20.68,19.00
4,2019-01-08,21.18,20.75,21.19,19.68
...,...,...,...,...,...
1253,2023-12-22,140.44,139.62,140.68,138.31
1254,2023-12-26,139.99,143.44,143.85,139.92
1255,2023-12-27,144.74,146.13,146.25,143.17
1256,2023-12-28,146.76,148.75,150.41,145.95


In [5]:
# This is a constant which will "Anualize" the volatility. there are around 252 trading days in a given year. 
#This is given by the space of a price interval, multiplied by how many intervals fit into the timespan we are converting the volatility to. 
#In this case interval = 1 day. 252 days fit into a year, which is what we want to convert the volatility to.
N = np.sqrt(252) 

#Now we need to convert from the prices to the returns, day to day.
percent_returns = df[["open","close", "high", "low"]].pct_change(1)
volatility_table = percent_returns.std() * N

if SHOW_INTERMEDIATE_DATA:
    display(percent_returns)
    display(volatility_table)

#This step will set the volatility we will use for later calculations, not sure which is best. Just made this as an arbitrary choice to have something.
volatility = volatility_table['open']
print(f"Calculated volatility: {volatility}")

drift_table = percent_returns.mean() * 252
drift = drift_table['open']
print(f"Calculated drift: {drift}")

,open,close,high,low
0,NaN,NaN,NaN,NaN
1,0.018837,-0.094049,-0.016842,-0.057842
2,-0.044589,0.114370,0.020878,0.028926
3,0.105862,0.082632,0.084426,0.090075
4,0.090067,0.008751,0.024662,0.035789
...,...,...,...,...
1253,0.017313,-0.001787,0.005001,0.006770
1254,-0.003204,0.027360,0.022533,0.011641
1255,0.033931,0.018753,0.016684,0.023228
1256,0.013956,0.017929,0.028444,0.019417


open     0.523952
close    0.537245
high     0.459543
low      0.465289
dtype: float64

Calculated volatility: 0.5239519629740491
Calculated drift: 0.5607745293243358


### Define the other option parameters:

In [6]:
# This is the current risk free rate of the market. I took mine from https://ycharts.com/indicators/10_year_treasury_rate (the end of 2023). 
# Make sure to change this depending on the time you want to apply the model at.
RISK_FREE_RATE = 0.0523

# Next we have all of the specifics for the option we would like to price:

# The strike price of the option, (in USD)
STRIKE_PRICE = 220

# The current price of the stock we would like to model.
CURRENT_STOCK_PRICE = 211.38

# Time until option expiry (make sure to stay consistent in units with volatility and risk_free_rate. In our case , we use days.
TIME_TILL_EXPIRY = 25

### Calculate the option price(s):

In [7]:
# This will use the montecarlo and blackscholes libraries we have built to return the expected stock price of an option with the parameters defined above.
# Will assume that it is a call option.
print(f"""
Estimated option prices:
Monte Carlo: ${mc(138.56, STRIKE_PRICE, TIME_TILL_EXPIRY/252, RISK_FREE_RATE, volatility, drift, timeSteps=1000, simulations=100000) + STRIKE_PRICE}
Black Scholes: ${bs(CURRENT_STOCK_PRICE, STRIKE_PRICE, RISK_FREE_RATE, volatility, TIME_TILL_EXPIRY) - STRIKE_PRICE}
""")


Estimated option prices:
Monte Carlo: $220.0657309753098
Black Scholes: $-28.483437582551147



## Using our fancier Monte Carlo:(broken, probably shouldn't use)

In [42]:
sample_returns = percent_returns["close"].to_numpy()[1:] # 1st number will alays be NaN

array([-0.09404888])

In [9]:
# hm(CURRENT_STOCK_PRICE, RISK_FREE_RATE,41 ,sample_returns, timeSteps=5000, simulations=10000)

In [10]:
# hm(138.56, RISK_FREE_RATE, 41,sample_returns, timeSteps = 1000, simulations=50000)


Timestep 0 completed
Timestep 100 completed
Timestep 200 completed
Timestep 300 completed
Timestep 400 completed
Timestep 500 completed
Timestep 600 completed
Timestep 700 completed
Timestep 800 completed
Timestep 900 completed


(168.89510765624973, 190.50577260224296, 126.46545780811832)

## Heston Model

Find correlation coefficient between asset return and volatility:

In [100]:
historical_returns = pd.concat([AMD_2019, AMD_2020]).reset_index()
historical_returns = historical_returns["close"].pct_change(1)[1:].to_numpy()

walking_volatility = np.vectorize(lambda n: np.concatenate([ historical_returns, sample_returns[:n] ],axis=0).std() * N)
sample_volatilities = np.fromfunction(walking_volatility, sample_returns.shape, dtype=int)
corr_coef = np.corrcoef(sample_volatilities, sample_returns)[0,1]

walking_vol_no_hist = np.vectorize(lambda n: sample_returns[:n+1].std() * N)
sample_no_hist_volatilities = np.fromfunction(walking_vol_no_hist, sample_returns.shape, dtype=int)
no_hist_corr = np.corrcoef(sample_no_hist_volatilities, sample_returns)[0,1]

print(f"Correlation coefficient: {corr_coef}")
print(f"With no \"historic\" data: {no_hist_corr}")
# sample_volatilities = #

Correlation coefficient: 0.026284351854126517
With no "historic" data: 0.09583035376904195


Find volatility of volatility: 

In [104]:
print(f"With historic data {sample_volatilities.std()}")
print(f"Without: {sample_no_hist_volatilities.std()}")

With historic data 0.014073697319694918
Without: 0.08548390178333708
